<a href="https://colab.research.google.com/github/stevenrdz/IR-SOES/blob/master/SOES.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install spacy
!python -m spacy download es_core_news_md

✔ Download and installation successful
You can now load the model via spacy.load('es_core_news_md')


In [2]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [3]:
from google.colab import files
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [4]:
from time import time
start_nb = time()

start = time()
import pandas as pd
from nltk import SnowballStemmer
from nltk.corpus import stopwords
import re
from nltk import word_tokenize

import es_core_news_md
nlp = es_core_news_md.load()

es_stops = set(stopwords.words('spanish'))
Snowball_stemmer = SnowballStemmer('spanish')

print('Importación de Librerias. %.2f segundos' % (time() - start))

Importación de Librerias. 18.09 segundos


# PRE-PROCESSED

### DATASET

In [5]:
start = time()
dataset = pd.read_csv('//content/gdrive/My Drive/df.csv',
                      sep=';', encoding='utf-8-sig')
dataset_size = len(dataset)


print('Dataset generado. %.2f segundos' % (time() - start))
dataset.head()

Dataset generado. 0.04 segundos


,Id,Title,Body
0,286498,Debugear código Django utilizando Docker,<p>Les cuento.\r\nEstoy realizando un proyecto...
1,226916,En el file.conf muestra Error usando Superviso...,<p>Al configurar el file.conf<br>\r\nruta : su...
2,161561,Ejecutar navegador en pantalla completa al ini...,<p>Estoy realizando un <em>proyecto Domotico</...
3,230560,Pagina web en blanco,<p>Buen dia \r\nNo se por que mi pagina web sa...
4,309146,Obtengo el error I/Choreographer: Skipped 110 ...,<p>Estoy trabajando en android studio intentan...


### LEMATIZACIÓN

In [6]:
def preprocess_lemma(doc):
    separator = ' '
    label = re.compile('<.*?>')
    doc = re.sub(label, '', doc)
    doc = doc.lower()  # Minuscula todo el texto.
    doc = word_tokenize(doc)  # Dividir en palabras.
    doc = [w for w in doc if not w in es_stops]  # Eliminar stopwords.
    doc = [w for w in doc if w.isalpha()]  # Eliminar numbers and punctuation.
    doc = separator.join(doc)
    doc = nlp(doc)
    doc = [w.lemma_ for w in doc]
    return doc

In [7]:
start = time()
dataset['Lemma_Title'] = dataset.apply(
        lambda dataset: preprocess_lemma(dataset['Title']), axis=1)
dataset['Lemma_Body'] = dataset.apply(
        lambda dataset: preprocess_lemma(dataset['Body']), axis=1)
print('Dataset generado. %.2f segundos' % (time() - start))
dataset.iloc[:,[3,4]]

Dataset generado. 27.11 segundos


,Lemma_Title,Lemma_Body
0,"[debugear, código, django, utilizar, docker]","[contar, realizar, proyectar, django, utilizar..."
1,"[mostrar, error, usar, supervisor, digital, oc...","[configurar, ruta, sudar, nano, program, factu..."
2,"[ejecutar, navegador, pantalla, completo, inic...","[realizar, proyectar, domotico, siguiente, com..."
3,"[paginar, web, blanco]","[bueno, dia, paginar, web, salir, blanco, algu..."
4,"[obtener, error, skipped, frames, the, applica...","[trabajar, android, studio, intentar, hacer, c..."
...,...,...
1079,"[conexion, remoto, sql, server, linux, mint]","[instalar, sql, server, linux, mint, basar, da..."
1080,"[crear, basar, dato, centos, funcionar]","[utilizar, centos, seguir, instrucción, sitiar..."
1081,"[hacer, respaldar, basar, dato, terminal]","[basar, dato, milisegundo, sql, server, expres..."
1082,"[aplicación, windows, linux, wine]","[aplicación, desarrollar, utilizar, cámara, es..."


### STEMMER

In [8]:
def preprocess_stem(doc):
    label = re.compile('<.*?>')
    doc = re.sub(label, '', doc)
    doc = doc.lower()  # Minuscula todo el texto.
    doc = word_tokenize(doc)  # Dividir en palabras.
    doc = [w for w in doc if not w in es_stops]  # Eliminar stopwords.
    doc = [w for w in doc if w.isalpha()]  # Eliminar numbers and punctuation.
    doc = [Snowball_stemmer.stem(w) for w in doc]  # Extraer raices o stems (Stemming)
    return doc

In [9]:
start = time()
dataset['Stem_Title'] = dataset.apply(
        lambda dataset: preprocess_stem(dataset['Title']), axis=1)
dataset['Stem_Body'] = dataset.apply(
        lambda dataset: preprocess_stem(dataset['Body']), axis=1)
print('Preprocesado completo. %.2f segundos' % (time() - start))
dataset.iloc[:,[5,6]]

Preprocesado completo. 3.40 segundos


,Stem_Title,Stem_Body
0,"[debug, codig, djang, utiliz, dock]","[cuent, realiz, proyect, djang, utiliz, pychar..."
1,"[muestr, error, usand, supervisor, digital, oc...","[configur, rut, sud, nan, program, facturaempr..."
2,"[ejecut, naveg, pantall, complet, inici, linux]","[realiz, proyect, domot, siguient, component, ..."
3,"[pagin, web, blanc]","[buen, dia, pagin, web, sal, blanc, algui, pod..."
4,"[obteng, error, skipp, fram, the, application,...","[trabaj, andro, studi, intent, hac, conexion, ..."
...,...,...
1079,"[conexion, remot, sql, serv, linux, mint]","[instal, sql, serv, linux, mint, bas, dat, cre..."
1080,"[cre, bas, dat, cent, funcion]","[utiliz, cent, segu, instruccion, siti, micros..."
1081,"[hac, respald, bas, dat, terminal]","[bas, dat, ms, sql, serv, express, ubuntu, adm..."
1082,"[aplic, windows, linux, win]","[aplic, desarroll, utiliz, cam, escan, lector,..."


### PRE-PROCESADO (SOLO STOPWORD & SIGNOS DE PUNTUACIÓN)

In [10]:
def preprocess(doc):
    label = re.compile('<.*?>')
    doc = re.sub(label, '', doc)
    doc = doc.lower()  # Minuscula todo el texto.
    doc = word_tokenize(doc)  # Dividir en palabras.
    doc = [w for w in doc if not w in es_stops]  # Eliminar stopwords.
    doc = [w for w in doc if w.isalpha()]  # Eliminar numbers and punctuation.
    return doc

In [11]:
start = time()
dataset['Processed_Title'] = dataset.apply(
        lambda dataset: preprocess(dataset['Title']), axis=1)
dataset['Processed_Body'] = dataset.apply(
        lambda dataset: preprocess(dataset['Body']), axis=1)
print('Preprocesado completo. %.2f segundos' % (time() - start))
dataset.iloc[:,[7,8]]

Preprocesado completo. 1.52 segundos


,Processed_Title,Processed_Body
0,"[debugear, código, django, utilizando, docker]","[cuento, realizando, proyecto, django, utilizo..."
1,"[muestra, error, usando, supervisor, digital, ...","[configurar, ruta, sudo, nano, program, factur..."
2,"[ejecutar, navegador, pantalla, completa, inic...","[realizando, proyecto, domotico, siguientes, c..."
3,"[pagina, web, blanco]","[buen, dia, pagina, web, sale, blanco, alguien..."
4,"[obtengo, error, skipped, frames, the, applica...","[trabajando, android, studio, intentando, hace..."
...,...,...
1079,"[conexion, remota, sql, server, linux, mint]","[instalado, sql, server, linux, mint, base, da..."
1080,"[crear, base, datos, centos, funciona]","[utilizando, centos, seguí, instrucciones, sit..."
1081,"[hacer, respaldo, base, datos, terminal]","[base, datos, ms, sql, server, express, ubuntu..."
1082,"[aplicación, windows, linux, wine]","[aplicación, desarrollada, utiliza, cámara, es..."


# WORD MOVER DISTANCE (WMD)

In [12]:
from sklearn.model_selection import train_test_split 

### WMD - LEMMA

In [13]:
train_l, test_l = train_test_split(dataset['Lemma_Title'], test_size = 0.20, shuffle = False)

print("(Lemma) Ejemplos usados para entrenar: ", len(train_l))
print("(Lemma) Ejemplos usados para test: ", len(test_l))

(Lemma) Ejemplos usados para entrenar:  867
(Lemma) Ejemplos usados para test:  217


In [14]:
test_l

867                 [backup, postgresql, error, password]
868     [ejecutar, sentenciar, postgresql, líneo, coma...
869                          [raspberry, pi, kali, linux]
870     [existir, alguno, alternativo, raspberry, pi, ...
871     [problema, instalar, drive, adaptador, usb, wi...
                              ...                        
1079         [conexion, remoto, sql, server, linux, mint]
1080              [crear, basar, dato, centos, funcionar]
1081            [hacer, respaldar, basar, dato, terminal]
1082                   [aplicación, windows, linux, wine]
1083    [wordpress, solicitar, servidor, usuario, ftp,...
Name: Lemma_Title, Length: 217, dtype: object

In [15]:
lemma_title = dataset['Lemma_Title'].to_list()

title = dataset['Title'].to_list()

In [16]:
from gensim.models import Word2Vec

model_l = Word2Vec(lemma_title, workers=3, size=100)

# Inicialización de WmdSimilarity.
from gensim.similarities import WmdSimilarity
num_best = 10
instancia_l = WmdSimilarity(test_l, model_l, num_best)
print(instancia_l)

WmdSimilarity<217 docs, 100 features>


In [17]:
start = time()

msj_l = 'Error al automatizar la instalación de Postgresql 9.4 en sistema embebido con Debian'
bus_l = preprocess_lemma(msj_l)

sim_l = instancia_l[bus_l]
print(len(sim_l))
print ('Cell took %.2f seconds to run.' %(time() - start))

10
Cell took 0.08 seconds to run.


In [18]:
print ('Busqueda: ', msj_l)
print("-------------------------------------------------------")
for i in range(num_best): # (num_best)
    print (title[sim_l[i][0]],'---> sim = %.4f' % sim_l[i][1])

Busqueda:  Error al automatizar la instalación de Postgresql 9.4 en sistema embebido con Debian
-------------------------------------------------------
grep combinado con echo ---> sim = 0.5410
Como ejecutar mas de un archivo que esta en un parámetro de jenkins ---> sim = 0.5379
problemas con la configuracion de GitLab, netbeans y linux ---> sim = 0.5302
¿Qué puerto TCP usa el programa ping? ---> sim = 0.5284
¿Cómo mostrar en pantalla solo los archivos de noviembre y diciembre para luego hacerle un grep? ---> sim = 0.5274
¿Como agregar varios puertos al mismo contenedor de Docker? ---> sim = 0.5259
Problema con git en local ---> sim = 0.5248
bash script mostrar información de procesos ---> sim = 0.5223
Error al conectar dispositivo HDMI . No input driver specified, ignoring this device ---> sim = 0.5214
¿Como verificar si una cadena ingresada se encuentra dentro de un rango numérico? ---> sim = 0.5212


### WMD - STEMMER

In [19]:
train_s, test_s = train_test_split(dataset['Stem_Title'], test_size = 0.20, shuffle = False)

print("(Stemm) Ejemplos usados para entrenar: ", len(train_s))
print("(Stemm) Ejemplos usados para test: ", len(test_s))

(Stemm) Ejemplos usados para entrenar:  867
(Stemm) Ejemplos usados para test:  217


In [20]:
stemm_title = dataset['Stem_Title'].to_list()

title = dataset['Title'].to_list()

In [21]:
from gensim.models import Word2Vec
# Train Word2Vec on all the restaurants.
model_s = Word2Vec(stemm_title, workers=3, size=100)

# Inicialización de WmdSimilarity.
from gensim.similarities import WmdSimilarity
num_best = 10
instancia_s = WmdSimilarity(test_s, model_s, num_best)
print(instancia_s)

WmdSimilarity<217 docs, 100 features>


In [22]:
start = time()

msj_s = 'Error al automatizar la instalación de Postgresql 9.4 en sistema embebido con Debian'
bus_s = preprocess_stem(msj_s)

sim_s = instancia_s[bus_s] 
print(len(sim_s))
print ('Cell took %.2f seconds to run.' %(time() - start))

10
Cell took 0.09 seconds to run.


In [23]:
print ('Busqueda: ', msj_s)
print("-------------------------------------------------------")
for i in range(num_best): # (num_best)
    print (title[sim_s[i][0]],'---> sim = %.4f' % sim_s[i][1])

Busqueda:  Error al automatizar la instalación de Postgresql 9.4 en sistema embebido con Debian
-------------------------------------------------------
¿Cómo mostrar en pantalla solo los archivos de noviembre y diciembre para luego hacerle un grep? ---> sim = 0.6087
¿como puedo solucionar esto kernel panic not syncing vfs unable en parrot? ---> sim = 0.6009
Cambiar permisos de carpetas compartidas con docker-compose ---> sim = 0.6009
Pantalla completa Kali Linux VirtualBox? ---> sim = 0.6005
Problema server vsftpd linux ---> sim = 0.5987
moodle apache centos 7 ---> sim = 0.5920
¿Como hago para reinstalar libbz2.so.1.0? ---> sim = 0.5680
problemas con la configuracion de GitLab, netbeans y linux ---> sim = 0.5473
Obtengo el error I/Choreographer: Skipped 110 frames! The application may be doing too much work on its main thread ---> sim = 0.5430
Problema con git en local ---> sim = 0.5376


### WMD - TEXT

In [24]:
def preprocess(doc):
    label = re.compile('<.*?>')
    doc = re.sub(label, '', doc)
    doc = doc.lower()  # Minuscula todo el texto.
    doc = word_tokenize(doc)  # Dividir en palabras.
    doc = [w for w in doc if not w in es_stops]  # Eliminar stopwords.
    doc = [w for w in doc if w.isalpha()]  # Eliminar numbers and punctuation.
    return doc

In [25]:
train_p, test_p = train_test_split(dataset['Processed_Title'], test_size = 0.20, shuffle = False)

print("Ejemplos usados para entrenar: ", len(train_p))
print("Ejemplos usados para test: ", len(test_p))

Ejemplos usados para entrenar:  867
Ejemplos usados para test:  217


In [26]:
test_p

867                 [backup, postgresql, error, password]
868     [ejecutar, sentencias, postgresql, línea, coma...
869                          [raspberry, pi, kali, linux]
870     [existe, alguna, alternativa, raspberry, pi, c...
871     [problema, instalar, drive, adaptador, usb, wi...
                              ...                        
1079         [conexion, remota, sql, server, linux, mint]
1080               [crear, base, datos, centos, funciona]
1081             [hacer, respaldo, base, datos, terminal]
1082                   [aplicación, windows, linux, wine]
1083    [wordpress, solicita, servidor, usuario, ftp, ...
Name: Processed_Title, Length: 217, dtype: object

In [27]:
processed_title = dataset['Processed_Title'].to_list()

title = dataset['Title'].to_list()

In [28]:
from gensim.models import Word2Vec
# Train Word2Vec on all the restaurants.
model_p = Word2Vec(processed_title, workers=3, size=100)

# Inicialización de WmdSimilarity.
from gensim.similarities import WmdSimilarity
num_best = 10
instancia_p = WmdSimilarity(train_p, model_p, num_best=10)
print(instancia_p)

WmdSimilarity<867 docs, 100 features>


# WMD - RESULTADO

In [76]:
start = time()

msj_p = 'Configurar QT Creator para depuración remota'
bus_p = preprocess(msj_p)

sim_p = instancia_p[bus_p]
print(len(sim_p))
print ('Cell took %.2f seconds to run.' %(time() - start))

10
Cell took 0.18 seconds to run.


In [77]:
print ('Busqueda: ', msj_p)
print("-------------------------------------------------------")
for i in range(num_best): # (num_best)
    print (title[sim_p[i][0]],'---> sim = %.4f' % sim_p[i][1])

Busqueda:  Configurar QT Creator para depuración remota
-------------------------------------------------------
Configurar QT Creator para depuración remota ---> sim = 1.0000
QT - Dibujar sobre QPainter o sobre QPixmap? ---> sim = 0.6252
Como configurar Zabbix ---> sim = 0.6252
¿Cómo configurar eclipse mosquitto en windows? ---> sim = 0.5817
Crear archivos ejecutables multiplataforma con QT y Netbeans ---> sim = 0.5376
Cómo configurar Qt para que ejecute leyendo los datos de un archivo txt ---> sim = 0.5302
Apache y NGINX configurar subdominio ---> sim = 0.5257
¿Como configurar bien netbeans y javafx? He intentado casi todo y sigo teniendo estos errores ---> sim = 0.5188
¿Que falta por configurar? IP estática en Ubuntu 16.04 ---> sim = 0.5152
configurar variables de memoria para Oracle 11 y 12 linux ---> sim = 0.4941


# VECTOR SPACE MODEL (VSM)

In [79]:
from collections import Counter
import numpy as np
import math

p_title = dataset['Processed_Title'].to_list()

N = len(dataset) # N = Cantidad de documentos
alpha = 0.3
DF = {}

for i in range(N):
  for w in p_title[i]:
      try:
        DF[w].add(i)
      except:
        DF[w] = {i}

for i in DF:
        DF[i] = len(DF[i])

total_vocab_size = len(DF)
total_vocab = [x for x in DF]

In [80]:
def doc_freq(DF, word):
    c = 0
    try:
        c = DF[word]
    except:
        pass
    return c

# VSM - RESULTADO

In [82]:
    start = time()

    doc = 0
    tf_idf_title = {}

    for i in range(N):
        tokens = p_body[i]
        counter = Counter(tokens + p_title[i])
        words_count = len(tokens + p_title[i])
        for token in np.unique(tokens):
            tf = counter[token] / words_count
            df = doc_freq(DF, token)
            idf = np.log((N + 1) / (df + 1))
            tf_idf_title[doc, token] = tf * idf
        doc += 1

    for i in tf_idf_title:
        tf_idf_title[i] *= alpha

    # D = Matriz TF-IDF de terminos de documentos
    D = np.zeros((N, total_vocab_size)) 
    for i in tf_idf_title:
        try:
            ind = total_vocab.index(i[1])
            D[i[0]][ind] = tf_idf_title[i]
        except:
            pass

    print('Matriz TF-IDF de terminos de documentos generada. %.2f segundos' % (time() - start))
    print(D[0])
    print(D[1])

Matriz TF-IDF de terminos de documentos generada. 1.27 segundos
[0.04843222 0.03346368 0.03879558 ... 0.         0.         0.        ]
[0. 0. 0. ... 0. 0. 0.]


In [83]:
def cosine_sim(a, b):
    cos_sim = np.dot(a, b)/(np.linalg.norm(a)*np.linalg.norm(b))
    return cos_sim

In [84]:
def print_doc(dataset, id):
    with pd.option_context('display.max_colwidth', 500):
        print(dataset.iloc[[id], [0,1]])

In [85]:
start = time()

def gen_vector(tokens):
        Q = np.zeros((len(total_vocab)))
        counter = Counter(tokens)
        words_count = len(tokens)
        for token in np.unique(tokens):
            tf = counter[token] / words_count
            df = doc_freq(DF, token)
            idf = math.log((N + 1) / (df + 1))
            try:
                ind = total_vocab.index(token)
                Q[ind] = tf * idf
            except:
                pass
        return Q

def cosine_similarity(k, query):
        tokens = preprocess(query)
        print("\nQuery:", query)
        d_cosines = []
        query_vector = gen_vector(tokens)
        for d in D:
            d_cosines.append(cosine_sim(query_vector, d))
        index_sort = np.array(d_cosines).argsort()[-k:][::-1]
        value_array = np.array(d_cosines)
        value_sort = np.sort(value_array)[-k:][::-1]
        res = []
        for i in range(k):
            res.append((index_sort[i], value_sort[i]))
        return res



In [86]:
results = cosine_similarity(5, "Configurar QT Creator para depuración remota")

print("\n")
for i in range(len(results)):
  print_doc(dataset, results[i][0])
  print("sim: ",results[i]) 
  print("-------------------------------------------------------")

print('Tiempo de ejecución: %.2f segundos' % (time() - start))


Query: Configurar QT Creator para depuración remota


         Id                                         Title
240  227010  Configurar QT Creator para depuración remota
sim:  (240, 0.4532126075018467)
-------------------------------------------------------
        Id                                                                    Title
596  49528  Cómo configurar Qt para que ejecute leyendo los datos de un archivo txt
sim:  (596, 0.2882178034325943)
-------------------------------------------------------
          Id                                                 Title
1066  122881  Como instalar el modulo QT Designer en Linux(Ubuntu)
sim:  (1066, 0.18813393665316802)
-------------------------------------------------------
         Id                                           Title
485  279342  ¿Cómo configurar eclipse mosquitto en windows?
sim:  (485, 0.16772771944835205)
-------------------------------------------------------
         Id                                         

# COSINE SIMILARITY


---



In [37]:
start = time()
data = pd.read_csv('//content/gdrive/My Drive/ultimate.csv',
                      sep=';', encoding='utf-8-sig')


print('Dataset generado. %.2f segundos' % (time() - start))
data.head()

Dataset generado. 0.07 segundos


,Unnamed: 0,Id,Title,Body,Lemma_Title,Lemma_Body,Stem_Title,Stem_Body,Processed_Title,Processed_Body
0,0,286498,Debugear código Django utilizando Docker,<p>Les cuento.\r\nEstoy realizando un proyecto...,debugear código django utilizar docker,contar realizar proyectar django utilizar pych...,debug codig djang utiliz dock,cuent realiz proyect djang utiliz pycharm wind...,debugear código django utilizando docker,cuento realizando proyecto django utilizo pych...
1,1,226916,En el file.conf muestra Error usando Superviso...,<p>Al configurar el file.conf<br>\r\nruta : su...,mostrar error usar supervisor digital ocean op...,configurar ruta sudar nano program facturaempr...,muestr error usand supervisor digital oce opci...,configur rut sud nan program facturaempres eje...,muestra error usando supervisor digital ocean ...,configurar ruta sudo nano program facturaempre...
2,2,161561,Ejecutar navegador en pantalla completa al ini...,<p>Estoy realizando un <em>proyecto Domotico</...,ejecutar navegador pantalla completo iniciar l...,realizar proyectar domotico siguiente componen...,ejecut naveg pantall complet inici linux,realiz proyect domot siguient component raspbe...,ejecutar navegador pantalla completa iniciar l...,realizando proyecto domotico siguientes compon...
3,3,230560,Pagina web en blanco,<p>Buen dia \r\nNo se por que mi pagina web sa...,paginar web blanco,bueno dia paginar web salir blanco alguien pod...,pagin web blanc,buen dia pagin web sal blanc algui podri ayud ...,pagina web blanco,buen dia pagina web sale blanco alguien podria...
4,4,309146,Obtengo el error I/Choreographer: Skipped 110 ...,<p>Estoy trabajando en android studio intentan...,obtener error skipped frames the application m...,trabajar android studio intentar hacer conexio...,obteng error skipp fram the application may be...,trabaj andro studi intent hac conexion maquin ...,obtengo error skipped frames the application m...,trabajando android studio intentando hacer con...


In [38]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd
import numpy as np
import string

titles_cosim = data['Processed_Title'].to_list()
id_index = data['Id'].to_list()
# Create the Document Term Matrix
vectorizer = CountVectorizer()

In [64]:
def print_res(data, id):
    with pd.option_context('display.max_colwidth', 500):
        print(data.iloc[[id], [2,1]])

In [63]:
# Create the Document Term Matrix
count_vectorizer = CountVectorizer()
sparse_matrix = count_vectorizer.fit_transform(titles_cosim)
doc_term_matrix = sparse_matrix.todense()
dtm = pd.DataFrame(doc_term_matrix,
                  columns=count_vectorizer.get_feature_names())

# COSINE SIMILARITY - RESULTADO

In [52]:
# Create the Query Matrix
query = preprocess("Configurar QT Creator para depuración remota")
print(query)
query_matrix = count_vectorizer.transform(query)
query_term_matrix = query_matrix.todense()
qtm = pd.DataFrame(query_term_matrix,
                  columns=count_vectorizer.get_feature_names())

['configurar', 'qt', 'creator', 'depuración', 'remota']


In [53]:
# Results and format
k = 5
cossim_matrix = cosine_similarity(qtm, dtm)
last = len(cossim_matrix)-1
exa = []
arg_sort = cossim_matrix[last].argsort()[-k:][::-1]
print(arg_sort)
value_sort = np.sort(cossim_matrix[last])[-k:][::-1]
print(value_sort)

[ 240 1079  816 1083  362]
[0.4472136  0.40824829 0.33333333 0.         0.        ]


In [54]:
for i in range(k):
    exa.append((arg_sort[i], value_sort[i]))

In [55]:
for i in range(len(exa)):
  print_res(data, exa[i][0])
  print("sim: ",exa[i])
  print("-------------------------------------------------------")

                                            Title      Id
240  Configurar QT Creator para depuración remota  227010
sim:  (240, 0.4472135954999579)
-------------------------------------------------------
                                            Title      Id
1079  Conexion remota de SQL Server linux mint 19  229945
sim:  (1079, 0.4082482904638631)
-------------------------------------------------------
                                                                   Title      Id
816  Poner timeout a ejecución en bash lanzado en máquina remota vía SSH  267148
sim:  (816, 0.3333333333333333)
-------------------------------------------------------
                                                                                     Title      Id
1083  Wordpress me solicita mi servidor usuario ftp y contraseña ftp para instalar plugins  216119
sim:  (1083, 0.0)
-------------------------------------------------------
                                              Title      Id
362  ¿Es 